# Library

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import copy
import time
from retry import retry
from explicit import waiter
from selenium.common.exceptions import StaleElementReferenceException

# Global Variables

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 105.0.5195
Get LATEST chromedriver version for 105.0.5195 google-chrome
Driver [C:\Users\ADMIN\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


In [42]:
wait_time = 2
load_time = 2

comment = {
    "Via": "",
    "Date Time": "",
    "Summary": "",
    "Full Content": "",
    "Rating": "",
}

record = {
    "URL": "",
    "Comments": []
}


# Functions

In [4]:
def scrollToPageEnd(driver):
    SCROLL_PAUSE_TIME = 0.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height    

In [45]:
def checkCommentSection():
    try:
        driver.find_element_by_xpath("//div[@class='lists list-reviews']")
    except:
        return False
    return True  
    
def crawlComments(url):
    cmt_list = []
    
    driver.get(url)
    time.sleep(load_time)
    
    if checkCommentSection()==False:
        return cmt_list
    else:
        # Load all comments by clicking "Show More" button.
        while True:
            try:
                driver.find_element_by_xpath("//div[@class='lists list-reviews']/div/div").click()
            except:
                break
                
        cmts = driver.find_elements_by_xpath("//div[@class='lists list-reviews']/div/ul/li[@class='review-item fd-clearbox ng-scope']")
        for elem in cmts:
            cmt = comment.copy()
            cmt['Via'] = elem.find_element_by_xpath('./div/div[2]/div[@class="ru-stats"]/a').text.replace("via ", "")
            cmt["Date Time"] = elem.find_element_by_xpath('./div/div[2]/div[@class="ru-stats"]/span').text
            cmt["Summary"] = elem.find_element_by_xpath('./div[2]/a').text
            cmt["Full Content"] = elem.find_element_by_xpath('./div[2]/div/span').text
            cmt["Rating"] = elem.find_element_by_xpath('./div/div[2]/div/span').text
            
            cmt_list.append(cmt)
    return cmt_list

# Testing

In [40]:
url = "https://www.foody.vn/da-nang/dookki-viet-nam-lau-buffet-tokpokki"
crawlComments(url)

[{'Via': 'Web',
  'Date Time': '8/2/2022 10:19',
  'Summary': 'Dookki***- Lẩu & Buffet Tokpokki - Bạch Đằng',
  'Full Content': 'Giá hiện tại của DOOKKI là bao nhiêu vậy ạ',
  'Rating': '5.0'},
 {'Via': 'iPhone',
  'Date Time': '17/2/2021 15:04',
  'Summary': 'dooki danang',
  'Full Content': 'ngon nhưng mình thấy là nên cho thơi gian ăn 2 tiếng là ok. tại khi mình đi ăn vs bạn bè cũng muốn cả ăn cả nói chuyện, chờ đồ ăn cx mất đi r 5-10 phút, mà nhiều khi đi ăn thịt hết mà chờ tới ăn xong chuẩn bị tính tiền thì nv mới tới thêm đồ ăn',
  'Rating': '5.0'},
 {'Via': 'MobileWeb',
  'Date Time': '15/2/2021 10:46',
  'Summary': 'Ổn',
  'Full Content': 'Ở đây có thể đặt bàn trước không ạ? Em nghe nói đồ ăn khá ngon',
  'Rating': '4.6'},
 {'Via': 'iPhone',
  'Date Time': '29/8/2020 0:43',
  'Summary': 'Dookki***- Lẩu & Buffet Tokpokki - Bạch Đằng',
  'Full Content': "Đồ ăn quán sẽ cực hợp ai thích ăn cay, thích tinh bột, đồ chiên rán. Topping lẩu khá nhiều với chủ đạo bánh gạo, chả cá, tôm, r

In [47]:
url = "https://www.foody.vn/da-nang/tra-sua-ngo"
crawlComments(url)

[]

# Create a Restaurant's Comment Record

In [46]:
urls = ["https://www.foody.vn/da-nang/dookki-viet-nam-lau-buffet-tokpokki"]
docs = []

for url in urls:
    rec = record.copy()
    rec['URL'] = url
    rec['Comments'] = crawlComments(url)
    
    docs.append(rec)
    
docs

[{'URL': 'https://www.foody.vn/da-nang/dookki-viet-nam-lau-buffet-tokpokki',
  'Comments': [{'Via': 'Web',
    'Date Time': '8/2/2022 10:19',
    'Summary': 'Dookki***- Lẩu & Buffet Tokpokki - Bạch Đằng',
    'Full Content': 'Giá hiện tại của DOOKKI là bao nhiêu vậy ạ',
    'Rating': '5.0'},
   {'Via': 'iPhone',
    'Date Time': '17/2/2021 15:04',
    'Summary': 'dooki danang',
    'Full Content': 'ngon nhưng mình thấy là nên cho thơi gian ăn 2 tiếng là ok. tại khi mình đi ăn vs bạn bè cũng muốn cả ăn cả nói chuyện, chờ đồ ăn cx mất đi r 5-10 phút, mà nhiều khi đi ăn thịt hết mà chờ tới ăn xong chuẩn bị tính tiền thì nv mới tới thêm đồ ăn',
    'Rating': '5.0'},
   {'Via': 'MobileWeb',
    'Date Time': '15/2/2021 10:46',
    'Summary': 'Ổn',
    'Full Content': 'Ở đây có thể đặt bàn trước không ạ? Em nghe nói đồ ăn khá ngon',
    'Rating': '4.6'},
   {'Via': 'iPhone',
    'Date Time': '29/8/2020 0:43',
    'Summary': 'Dookki***- Lẩu & Buffet Tokpokki - Bạch Đằng',
    'Full Content': "